GLiNER to model rozpoznawania nazwanych encji (NER), który potrafi identyfikować dowolny typ encji za pomocą dwukierunkowego enkodera transformatorowego (podobnego do BERT-a). Stanowi praktyczną alternatywę dla tradycyjnych modeli NER, które są ograniczone do z góry określonych encji, oraz dla dużych modeli językowych (LLM), które mimo swojej elastyczności są kosztowne i zbyt duże w przypadku scenariuszy z ograniczonymi zasobami.

https://github.com/urchade/GLiNER <br>
https://huggingface.co/urchade/gliner_medium-v2.1 <br>
https://huggingface.co/urchade/gliner_large-v2

In [6]:
import pandas as pd
from gliner import GLiNER

In [7]:
model_name = "urchade/gliner_large-v2"
model = GLiNER.from_pretrained(model_name)
model.eval()
print("ok")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

c:\Users\racha\Desktop\PROJECTS\PKO\analysis-advanced-chunking-techniques\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


ok


Threshold w GLiNER (ang. confidence threshold) to minimalna wartość pewności modelu, powyżej której prognoza (dana etykieta) zostanie uznana za prawidłową i zwrócona w wynikach.

In [8]:
text = """
1. Użyte w Regulaminie określenia oznaczają:
1) dopuszczalne saldo debetowe – ujemne saldo rachunku oszczędnościowo-rozliczeniowego w wysokości nieprzekraczającej miesięcznego limitu ustalanego
przez PKO Bank Polski SA, z którego Posiadacz rachunku może korzystać na podstawie umowy rachunku,
2) data waluty – dzień, od którego lub do którego, naliczane są odsetki od środków pieniężnych, którymi został obciążony lub uznany rachunek,
3) dyspozycja – wszelkie oświadczenia woli lub wiedzy, w tym dotyczące operacji, składane przez Klienta,
4) dzień roboczy – dzień przypadający od poniedziałku do piątku, który nie jest dniem ustawowo wolnym od pracy,
5) kapitalizacja – dopisanie do salda rachunku naliczonych za dany okres odsetek od środków pieniężnych zgromadzonych na tym rachunku
"""

labels = [
    "kredyt hipoteczny", "pożyczka gotówkowa", "konto osobiste", "konto firmowe", "konto oszczędnościowe", "lokata terminowa",
    "ubezpieczenie", "kredyt firmowy", "kredyt konsolidacyjny", "karta kredytowa", "karta debetowa", "bankomat",
    "oddział bankowy", "opłaty", "prowizja", "transakcje", "limit kredytowy", "umowa kredytowa", "oprocentowanie",
    "kapitał", "odsetki", "raty", "czyszczenie historii kredytowej", "ubezpieczenie nieruchomości", "ubezpieczenie na życie",

]

entities = model.predict_entities(text, labels, threshold=0.4, multi_label=True)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


rachunku oszczędnościowo-rozliczeniowego => konto oszczędnościowe
miesięcznego limitu => limit kredytowy
PKO Bank Polski SA => bankomat
Posiadacz rachunku => konto osobiste
umowy rachunku => umowa kredytowa
odsetki => odsetki
operacji => transakcje
Klienta => konto osobiste
kapitalizacja => kapitał
odsetek => odsetki


In [15]:
chunks = [
"""
1. Użyte w Regulaminie określenia oznaczają:
1) dopuszczalne saldo debetowe – ujemne saldo rachunku oszczędnościowo-rozliczeniowego w wysokości nieprzekraczającej miesięcznego limitu ustalanego
przez PKO Bank Polski SA, z którego Posiadacz rachunku może korzystać na podstawie umowy rachunku,
2) data waluty – dzień, od którego lub do którego, naliczane są odsetki od środków pieniężnych, którymi został obciążony lub uznany rachunek,
3) dyspozycja – wszelkie oświadczenia woli lub wiedzy, w tym dotyczące operacji, składane przez Klienta,
4) dzień roboczy – dzień przypadający od poniedziałku do piątku, który nie jest dniem ustawowo wolnym od pracy,
5) kapitalizacja – dopisanie do salda rachunku naliczonych za dany okres odsetek od środków pieniężnych zgromadzonych na tym rachunku
""",
"""
1. PKO Bank Polski SA może prowadzić na rzecz Klientów rachunki bankowe, o których mowa w § 1. Rachunki te nie są przeznaczone do przeprowadzania rozliczeń
pieniężnych związanych z prowadzoną przez Klienta działalnością gospodarczą lub zarobkową.
2. PKO Bank Polski SA może otwierać kolejne rachunki na podstawie umowy o prowadzenie rachunków bankowych.
3. Zawarcie umowy rachunku może nastąpić:
1) za pośrednictwem elektronicznych kanałów dostępu, o ile PKO Bank Polski SA dla danego kanału udostępnia odpowiednią funkcjonalność,
2) w placówce,
3) w inny uzgodniony sposób.
4. Rachunek może być otwarty:
1) jako rachunek indywidualny - dla rezydenta albo nierezydenta,
2) jako rachunek wspólny – dla dwóch osób o tym samym statusie dewizowym, posiadających pełną zdolność do czynności prawnych, które posiadają ten
sam kraj rezydencji podatkowej i obie legitymują się ważnymi certyfikatami rezydencji podatkowej, bądź obie nie posiadają ważnych certyfikatów rezydencji
podatkowej
""",
"""
1. Posiadacz rachunku, który nie ukończył 13 lat, albo osoba ubezwłasnowolniona całkowicie nie może samodzielnie dysponować środkami pieniężnymi
zgromadzonymi na rachunku, ani dokonywać samodzielnie żadnych czynności związanych z posiadaniem rachunku.
2. W imieniu Posiadacza rachunku, o którym mowa w ust. 1, do dysponowania środkami pieniężnymi zgromadzonymi na rachunku w granicach zwykłego zarządu,
uprawniony jest jego przedstawiciel ustawowy, który zawarł umowę rachunku w imieniu Posiadacza rachunku. Dysponowanie środkami pieniężnymi w zakresie
przekraczającym zwykły zarząd, wymaga zezwolenia sądu opiekuńczego
""",
"""
1. Za usługi świadczone przez PKO Bank Polski SA, w tym za czynności związane z prowadzeniem rachunku, wysyłaniem powiadomień, a także za inne czynności,
PKO Bank Polski SA pobiera prowizje i opłaty bankowe, zgodnie z Taryfą.
2. PKO Bank Polski SA należne prowizje i opłaty bankowe, o których mowa w ust. 1, pobierać będzie, z uwzględnieniem ust. 3-4, w ciężar rachunku, za wyjątkiem
rachunku lokaty.
3. W przypadku rachunku lokaty należne prowizje i opłaty bankowe PKO Bank Polski SA pobierać będzie z rachunku, o którym mowa w § 25 ust. 2 albo w formie
gotówkowej.
4. W przypadku braku wolnych środków na rachunku, pobranie:
1) prowizji i opłat, lub
2) zaliczek na podatek dochodowy oraz składek na ubezpieczenie zdrowotne od realizowanych przez PKO Bank Polski SA rent i emerytur z zagranicy
może spowodować powstanie zadłużenia przeterminowanego.
""",
"""
1. Środki pieniężne zgromadzone na rachunku mogą być oprocentowane zgodnie z zasadami określonymi w Regulaminie lub w umowie rachunku.
2. W przypadku, gdy środki pieniężne zgromadzone na rachunku są oprocentowane według zmiennej stopy procentowej, w okresie obowiązywania umowy
rachunku, PKO Bank Polski SA jest uprawniony do zmiany wysokości oprocentowania środków pieniężnych zgromadzonych na tym rachunku na zasadach
określonych w ust. 3.
""",
"""Oddział naszego banku zlokalizowany przy ulicy Warszawskiej 12 oferuje pełen zakres usług finansowych dla klientów indywidualnych i firmowych.
Klienci mogą tam skorzystać z doradztwa kredytowego oraz założyć rachunek oszczędnościowy.
""",
"""Oddział bankowy zyskał na znaczeniu!
"""
]

In [16]:
entities = model.batch_predict_entities(chunks, labels, multi_label=True)

In [17]:
entities

[[{'start': 92,
   'end': 132,
   'text': 'rachunku oszczędnościowo-rozliczeniowego',
   'label': 'konto oszczędnościowe',
   'score': 0.6408064365386963},
  {'start': 164,
   'end': 183,
   'text': 'miesięcznego limitu',
   'label': 'limit kredytowy',
   'score': 0.8490257859230042},
  {'start': 201,
   'end': 219,
   'text': 'PKO Bank Polski SA',
   'label': 'bankomat',
   'score': 0.9385367631912231},
  {'start': 278,
   'end': 292,
   'text': 'umowy rachunku',
   'label': 'umowa kredytowa',
   'score': 0.6764527559280396},
  {'start': 358,
   'end': 365,
   'text': 'odsetki',
   'label': 'odsetki',
   'score': 0.5965433716773987},
  {'start': 507,
   'end': 515,
   'text': 'operacji',
   'label': 'transakcje',
   'score': 0.9082099199295044},
  {'start': 656,
   'end': 669,
   'text': 'kapitalizacja',
   'label': 'kapitał',
   'score': 0.5334851741790771},
  {'start': 726,
   'end': 733,
   'text': 'odsetek',
   'label': 'odsetki',
   'score': 0.7038186192512512}],
 [{'start': 4,
 

In [18]:
results = []
for chunk, entity_list in zip(chunks, entities):
    matched_labels = list(set(entity['label'] for entity in entity_list)) 
    results.append({"Chunk": chunk, "Matched Labels": matched_labels})

results_df = pd.DataFrame(results)

results_df

,Chunk,Matched Labels
0,\n1. Użyte w Regulaminie określenia oznaczają:...,"[bankomat, kapitał, odsetki, limit kredytowy, ..."
1,\n1. PKO Bank Polski SA może prowadzić na rzec...,"[bankomat, umowa kredytowa, oddział bankowy]"
2,"\n1. Posiadacz rachunku, który nie ukończył 13...","[konto osobiste, konto firmowe, umowa kredytowa]"
3,\n1. Za usługi świadczone przez PKO Bank Polsk...,"[bankomat, pożyczka gotówkowa, ubezpieczenie, ..."
4,\n1. Środki pieniężne zgromadzone na rachunku ...,"[bankomat, oprocentowanie, karta kredytowa, ko..."
5,Oddział naszego banku zlokalizowany przy ulicy...,"[bankomat, konto oszczędnościowe, konto firmowe]"
6,Oddział bankowy zyskał na znaczeniu!\n,[oddział bankowy]


In [19]:
print(results_df)

                                               Chunk  \
0  \n1. Użyte w Regulaminie określenia oznaczają:...   
1  \n1. PKO Bank Polski SA może prowadzić na rzec...   
2  \n1. Posiadacz rachunku, który nie ukończył 13...   
3  \n1. Za usługi świadczone przez PKO Bank Polsk...   
4  \n1. Środki pieniężne zgromadzone na rachunku ...   
5  Oddział naszego banku zlokalizowany przy ulicy...   
6             Oddział bankowy zyskał na znaczeniu!\n   

                                      Matched Labels  
0  [bankomat, kapitał, odsetki, limit kredytowy, ...  
1       [bankomat, umowa kredytowa, oddział bankowy]  
2   [konto osobiste, konto firmowe, umowa kredytowa]  
3  [bankomat, pożyczka gotówkowa, ubezpieczenie, ...  
4  [bankomat, oprocentowanie, karta kredytowa, ko...  
5   [bankomat, konto oszczędnościowe, konto firmowe]  
6                                  [oddział bankowy]  
